In [ ]:
from bs4 import BeautifulSoup
from dataclasses import dataclass, asdict
from typing import List
import yaml
import requests
import json
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
def read_config(config_path: str):
    with open(config_path, "r") as file:
        yaml_content = yaml.safe_load(file)
    return yaml_content

In [ ]:
parameters_path = "./config/query_params.yaml"
scraper_path = "./config/scraper.yaml"

In [ ]:
scraper_config = read_config(parameters_path)

In [ ]:
scraper_config

In [ ]:
scraper_config['price_min']

In [ ]:
url = 'https://www.otodom.pl/pl/oferta/nowolipki-3-pokoje-63-2m2-ID4oITS'
# url = 'https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-54m2-2-balkony-ID4nOyG'

In [ ]:
# one_add_page = f'https://www.otodom.pl{one_url}'
one_add_page = url
page_one_add = requests.get(one_add_page, timeout=2)
soup_one_add = BeautifulSoup(page_one_add.content, 'html.parser')
soup_one_add.find_all("div", {"class": "css-m97llu e16xl7020"})
json_data = json.loads(soup_one_add.find('script', type='application/json').text)
json_data = json_data['props']['pageProps']

In [ ]:
def extract_ad_main_char(json_data: dict, scraper_config: dict):
    paresed_gen = {}
    for c in scraper_config['general_ad_attr']:
        ads_gen = {c: json_data.get(c, {})}
        paresed_gen.update(ads_gen)
    return paresed_gen

def extract_ad_details(json_data: dict, scraper_config: dict):
    parsed_details = {}
    add_details_json = json_data.get("ad", {})
    for c in scraper_config['ad_attr']:
        ad_details = {c: add_details_json.get(c, {})}
        parsed_details.update(ad_details)
    return parsed_details

def extract_target_details(json_data: dict, scraper_config: dict):
    parsed_targets = {}
    add_target_json = json_data.get("ad", {}).get("target", {})
    for c in scraper_config['ad_targets_attr']:
        ad_targets = {c: add_target_json.get(c, {})}
        parsed_targets.update(ad_targets)
    return parsed_targets

def extract_ad_geo(json_data: dict, scraper_config: dict):
    parsed_geo = {}
    add_geo_json = json_data.get("ad", {}).get("location", {}).get('coordinates', {})
    for c in scraper_config['geo_attr']:
        ad_geo = {c: add_geo_json.get(c, {})}
        parsed_geo.update(ad_geo)
    return parsed_geo

def extract_ad_address(json_data: dict, scraper_config: dict):
    address_parsed = {}
    ad_address_json = json_data.get("ad", {}).get("location", {}).get('address', {})
    for c in scraper_config['address_attr']:        
        ad_address = {
            f'{c}_id': ad_address_json.get(c, {}).get('id', {}),
            f'{c}_name': ad_address_json.get(c, {}).get('name', {}),
        }
        address_parsed.update(ad_address)
    return address_parsed

def full_ad_details(json_data: dict, scraper_config: dict):
    full_data = {}
    full_data.update(extract_ad_main_char(json_data, scraper_config))
    full_data.update(extract_ad_details(json_data, scraper_config))
    full_data.update(extract_target_details(json_data, scraper_config))
    full_data.update(extract_ad_geo(json_data, scraper_config))
    full_data.update(extract_ad_address(json_data, scraper_config))
    return full_data

In [ ]:
extract_target_details(json_data = json_data, scraper_config = scraper_config)

In [ ]:
full_ad_details(json_data = json_data, scraper_config = scraper_config)

In [ ]:
import os

In [ ]:
os.path.isdir('new_folder')

In [ ]:
os.path.exists(os.path.join(os.getcwd(), 'batch'))

In [1]:
from src.save_OtoData import create_save_data_path, save_file

In [3]:
create_save_data_path(False)

'C:\\Users\\Maciej Baranowski\\Mój dysk\\Colab Notebooks\\Praca_Zaliczeniowa\\full_df/oto_dom_full_20240126_170933.pkl'